In [60]:
from data_raw import raw_df as raw_df
import pandas as pd
import numpy as np

The link I'm using for this project is from a Kaggle Dataset [Cafe Sales Dirty Data](https://www.kaggle.com/datasets/ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training/data). Before diving into the data, we have some information:
* Some columns (e.g., Item, Payment Method, Location) may contain missing values represented as None or empty cells.
* Some rows contain invalid entries like "ERROR" or "UNKNOWN"
* Prices for menu items are consistent but may have missing or incorrect values introduced Below is the data table of the correct values for the prices:

In [61]:
prices_dict = {"Item": ["Coffee", "Tea", "Sandwich", "Salad", "Cake", "Cookie", "Smoothie", "Juice"],
             "Price Per Unit": [2,1.5,4,5,3,1,4,3]}
prices_df = pd.DataFrame(prices_dict)
prices_df

Item  Price Per Unit
0    Coffee             2.0
1       Tea             1.5
2  Sandwich             4.0
3     Salad             5.0
4      Cake             3.0
5    Cookie             1.0
6  Smoothie             4.0
7     Juice             3.0

In [65]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8611 entries, 0 to 8610
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Transaction ID     8611 non-null   object 
 1   Item               8611 non-null   object 
 2   Quantity           8611 non-null   object 
 3   Price Per Unit     8147 non-null   object 
 4   Total Spent        8179 non-null   object 
 5   Payment Method     5879 non-null   object 
 6   Location           5193 non-null   object 
 7   Transaction Date   8215 non-null   object 
 8   Price Per UnitPPU  8611 non-null   float64
dtypes: float64(1), object(8)
memory usage: 605.6+ KB


Transaction ID is the only column which has no missing values. This tracks with the information that came with the data, but we should check that each transaction has a unique ID.

In [12]:
raw_df.nunique()

Transaction ID      10000
Item                   10
Quantity                7
Price Per Unit          8
Total Spent            19
Payment Method          5
Location                4
Transaction Date      367
dtype: int64

This confirms that each has a unique ID. Now, let's change all of the ERROR, or UNKNOWN values to NA.

In [52]:
raw_df.replace(["UNKNOWN", "ERROR"], np.nan, inplace=True)

In [64]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8611 entries, 0 to 8610
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Transaction ID     8611 non-null   object 
 1   Item               8611 non-null   object 
 2   Quantity           8611 non-null   object 
 3   Price Per Unit     8147 non-null   object 
 4   Total Spent        8179 non-null   object 
 5   Payment Method     5879 non-null   object 
 6   Location           5193 non-null   object 
 7   Transaction Date   8215 non-null   object 
 8   Price Per UnitPPU  8611 non-null   float64
dtypes: float64(1), object(8)
memory usage: 605.6+ KB


Now we can see that lots more of the data, was NA than we knew. Let's convert the types of values in the dataframe and make sure that all of the items are marked for the right price. If the item is saved, we can input the correct price into the price per unit column and use that and the quantity to make sure that the total spent is correct. However, since we know that the Price Per Unit column has errors, we cannot reverse engineer this process. With that being said, we cannot confirm the item or quantity based on other data, so we can drop any rows where those are missing values.

In [37]:
raw_df.dropna(subset = ["Item", "Quantity"], inplace = True)

In [63]:
raw_df = raw_df.merge(prices_df, on = "Item", how = "left", suffixes = ("", "PPU"))
raw_df

Transaction ID      Item Quantity Price Per Unit Total Spent  \
0       TXN_1961373    Coffee        2            2.0         4.0   
1       TXN_4977031      Cake        4            3.0        12.0   
2       TXN_4271903    Cookie        4            1.0         NaN   
3       TXN_7034554     Salad        2            5.0        10.0   
4       TXN_3160411    Coffee        2            2.0         4.0   
...             ...       ...      ...            ...         ...   
8606    TXN_4766549  Smoothie        2            4.0         NaN   
8607    TXN_7672686    Coffee        2            2.0         4.0   
8608    TXN_5255387    Coffee        4            2.0         8.0   
8609    TXN_7695629    Cookie        3            NaN         3.0   
8610    TXN_6170729  Sandwich        3            4.0        12.0   

      Payment Method  Location Transaction Date  Price Per UnitPPU  
0        Credit Card  Takeaway       2023-09-08                2.0  
1               Cash  In-store       2023-05-16                3.0  
2        Credit Card  In-store       2023-07-19                1.0  
3                NaN       NaN       2023-04-27                5.0  
4     Digital Wallet  In-store       2023-06-11                2.0  
...              ...       ...              ...                ...  
8606            Cash       NaN       2023-10-20                4.0  
8607             NaN       NaN       2023-08-30                2.0  
8608  Digital Wallet       NaN       2023-03-02                2.0  
8609  Digital Wallet       NaN       2023-12-02                1.0  
8610            Cash  In-store       2023-11-07                4.0  

[8611 rows x 9 columns]